In [ ]:
from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.vectorstores import FAISS
from langchain.embeddings import HuggingFaceEmbeddings
from langchain.chains import ConversationalRetrievalChain
from langchain.llms import HuggingFacePipeline
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
import bitsandbytes as bnb

In [ ]:
# Step 1: Load the IPC Document
pdf_path = "MyDrive/ipc_act.pdf"  
document_loader = PyPDFLoader(pdf_path)
documents = document_loader.load()

In [5]:
# Step 2: Process and Split the Text
text_splitter = RecursiveCharacterTextSplitter(chunk_size=1000, chunk_overlap=200)
split_docs = text_splitter.split_documents(documents)

In [6]:
# Step 3: Create a FAISS Vector Store (using CPU version, optimized for memory)
embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
vector_store = FAISS.from_documents(split_docs, embeddings)
retriever = vector_store.as_retriever()

<ipython-input-6-c68b1abc7b24>:2: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(model_name="sentence-transformers/all-MiniLM-L6-v2")
/usr/local/lib/python3.11/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or da

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/10.5k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [8]:
!huggingface-cli login


    _|    _|  _|    _|    _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|_|_|_|    _|_|      _|_|_|  _|_|_|_|
    _|    _|  _|    _|  _|        _|          _|    _|_|    _|  _|            _|        _|    _|  _|        _|
    _|_|_|_|  _|    _|  _|  _|_|  _|  _|_|    _|    _|  _|  _|  _|  _|_|      _|_|_|    _|_|_|_|  _|        _|_|_|
    _|    _|  _|    _|  _|    _|  _|    _|    _|    _|    _|_|  _|    _|      _|        _|    _|  _|        _|
    _|    _|    _|_|      _|_|_|    _|_|_|  _|_|_|  _|      _|    _|_|_|      _|        _|    _|    _|_|_|  _|_|_|_|

    To log in, `huggingface_hub` requires a token generated from https://huggingface.co/settings/tokens .
Enter your token (input will not be visible): 
Add token as git credential? (Y/n) n
Token is valid (permission: fineGrained).
The token `try` has been saved to /root/.cache/huggingface/stored_tokens
Your token has been saved to /root/.cache/huggingface/token
Login successful.
The current active token is: `try`


In [9]:
# Step 4: Load a Smaller Free LLM from Hugging Face (Optimized for RAM)
model_name = "meta-llama/Llama-3.1-8B-Instruct"#"open-thoughts/OpenThinker-7B"  # Smaller and optimized model
tokenizer = AutoTokenizer.from_pretrained(model_name)
model = AutoModelForCausalLM.from_pretrained(
    model_name,
    torch_dtype=torch.float16,
    device_map="auto",
    load_in_4bit=True  # Enables 4-bit quantization for speedup
)
nlp_pipeline = pipeline(
    "text-generation",
    model=model,
    tokenizer=tokenizer,
    max_new_tokens=100,
    temperature=0.7,
    repetition_penalty=1.1
)


llm = HuggingFacePipeline(pipeline=nlp_pipeline)


tokenizer_config.json:   0%|          | 0.00/55.4k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/296 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/855 [00:00<?, ?B/s]

The `load_in_4bit` and `load_in_8bit` arguments are deprecated and will be removed in the future versions. Please, pass a `BitsAndBytesConfig` object in `quantization_config` argument instead.


...

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/184 [00:00<?, ?B/s]

Device set to use cuda:0
<ipython-input-9-7712c2b8b6d0>:20: LangChainDeprecationWarning: The class `HuggingFacePipeline` was deprecated in LangChain 0.0.37 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFacePipeline``.
  llm = HuggingFacePipeline(pipeline=nlp_pipeline)


In [10]:
# Step 5: Build the Conversational Chain
qa_chain = ConversationalRetrievalChain.from_llm(llm, retriever)

In [11]:
def chat():
    print("IPC Chatbot: Ask me about the Indian Penal Code!")
    chat_history = []
    while True:
        query = input("You: ")
        if query.lower() in ['exit', 'quit', 'bye']:
            print("IPC Chatbot: Goodbye!")
            break
        response = qa_chain.invoke({"question": query, "chat_history": chat_history})
        chat_history.append((query, response['answer']))
        print("IPC Chatbot:", response['answer'])
chat()

IPC Chatbot: Ask me about the Indian Penal Code!
You: What's The punishment for murder


Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
/usr/local/lib/python3.11/dist-packages/bitsandbytes/nn/modules.py:451: UserWarning: Input type into Linear4bit is torch.float16, but bnb_4bit_compute_dtype=torch.float32 (default). This will lead to slow inference or training speed.
  warnings.warn(


IPC Chatbot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

himself to he likely to cause. 
302. Punishment for murder .—Whoever commits murder sha ll be punished with death or 
1[imprisonment for life], and shall also be liable to fine. 
303. Punishment for murder by life -convict.—Whoever, being under sentence of 1[imprisonment 
for life], commits murder, shall be punished with death. 
304. Punishment for culpable homi cide not amounting to murder .—Whoever commits culpable 
homicide not amounting to murder, shall be punished with 1[imprisonment for life], or imprisonment of 
either description for a term which may extend to ten years, and shall also be liable to fine, if the act by 
which the death is caused is done with the intention of causing death , or of causing such bodily inju ry as 
is likely to cause death; 
or with imprisonment of either description for a te

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


IPC Chatbot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

himself to he likely to cause. 
302. Punishment for murder .—Whoever commits murder sha ll be punished with death or 
1[imprisonment for life], and shall also be liable to fine. 
303. Punishment for murder by life -convict.—Whoever, being under sentence of 1[imprisonment 
for life], commits murder, shall be punished with death. 
304. Punishment for culpable homi cide not amounting to murder .—Whoever commits culpable 
homicide not amounting to murder, shall be punished with 1[imprisonment for life], or imprisonment of 
either description for a term which may extend to ten years, and shall also be liable to fine, if the act by 
which the death is caused is done with the intention of causing death , or of causing such bodily inju ry as 
is likely to cause death; 
or with imprisonment of either description for a te

Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


IPC Chatbot: Use the following pieces of context to answer the question at the end. If you don't know the answer, just say that you don't know, don't try to make up an answer.

himself to he likely to cause. 
302. Punishment for murder .—Whoever commits murder sha ll be punished with death or 
1[imprisonment for life], and shall also be liable to fine. 
303. Punishment for murder by life -convict.—Whoever, being under sentence of 1[imprisonment 
for life], commits murder, shall be punished with death. 
304. Punishment for culpable homi cide not amounting to murder .—Whoever commits culpable 
homicide not amounting to murder, shall be punished with 1[imprisonment for life], or imprisonment of 
either description for a term which may extend to ten years, and shall also be liable to fine, if the act by 
which the death is caused is done with the intention of causing death , or of causing such bodily inju ry as 
is likely to cause death; 
or with imprisonment of either description for a te